In [6]:
# function pred_img(img_in, model_info; batch_size = 1)
#     # get binary mask
#     img = deepcopy(img_in)
#     img_out = zeros(Float32, size(img))
#     img_mask = 1 .- round.(zoom_pxiel_values(img_in))
    
#     # patch img_in
#     img_patches, locations, num_patches = patch_image(img, img_mask) # size of img_patches = (256, 256, 1, x)
#     pred_patches = Array{Float32, 3}(undef, patch_size, patch_size, num_patches)
#     temp = floor(Int, num_patches/4)
#     num_patches_each_gpu = [temp, temp, temp, num_patches - 3*temp]

#     # Distribute to 4 GPUs
#     @Threads.threads for gpu_idx = 1:4
#         device!(gpu_idx-1)
#         # load model
#         model_, ps_, st_ = model_info
#         model = deepcopy(model_)
#         ps = ps_ |> gpu 
#         st = st_ |> gpu 

#         # distribute patch
#         curr_start = sum(num_patches_each_gpu[1 : gpu_idx-1]) + 1
#         curr_end = curr_start + num_patches_each_gpu[gpu_idx] - 1
#         # for img_idx = curr_start : batch_size : curr_end
#             # idx_start = img_idx
#             # idx_end = min(curr_end, idx_start + batch_size - 1)
#             idx_start = curr_start
#             idx_end = curr_end
#             @info "GPU $gpu_idx: $idx_start to $idx_end = $(idx_end-idx_start+1)"
#             ignore_derivatives() do
#                 x = img_patches[:, :, :, idx_start : idx_end]          # 256*256*1*batch_size
#                 curr_pred = model(x |> gpu , ps, st)[1] |> cpu         # 256*256*1*batch_size  
#                 pred_patches[:,:,idx_start : idx_end] = curr_pred[:,:,1,:]
#             #     CUDA.unsafe_free!(curr_pred_)
#             #     CUDA.unsafe_free!(x)
#             end
#         # end
#         # ps, st = nothing, nothing
#         GC.gc(true)
#         CUDA.reclaim()
#         CUDA.memory_status()
#     end

#     # combines patch to pred_out
#     Threads.@threads for i = 1 : num_patches
#         x_start, x_end, y_start, y_end = locations[i]
#         for x = x_start : x_end
#             for y = y_start : y_end
#                 img_out[x,y] = pred_patches[x-x_start+1,y-y_start+1,i]>=0.5 ? 1f0 : img_out[x,y]
#                 # img_out[x,y] += pred_patches[x-x_start+1,y-y_start+1,i]
#             end
#         end
#     end
#     return img_out
# end

# 2. Save predictions as png

In [48]:
plot_SIDs = readdir(output_pred_dir)
for SID in plot_SIDs
    curr_dir = joinpath(output_pred_dir, SID)
    if isdir(curr_dir)
        for f in readdir(curr_dir)
            # read
            curr_path = joinpath(curr_dir, f)
            @load curr_path pred_image label
            # convert to png
            image_output = Gray.(zoom_pxiel_values(pred_image))
            label_output = Gray.(label)
            # set up path
            f_name, _ = splitext(f)
            image_path = joinpath(output_images_dir, SID, "pred_"*f_name*".png")
            label_path = joinpath(output_images_dir, SID, "roi_"*f_name*".png")
            # save
            save(image_path, image_output)
            save(label_path, label_output)
        end
    end
end
GC.gc(true)

# 3. Plot

In [5]:
plot_SIDs = readdir(output_pred_dir)
areas_ = []
for SID in plot_SIDs
    curr_dir = joinpath(output_pred_dir, SID)
    if isdir(curr_dir)
        for f in readdir(curr_dir)
            curr_path = joinpath(curr_dir, f)
            @load curr_path pred_image label
            push!(areas_, [SID, f, sum(label), sum(pred_image)])
        end
    end
end
s = size(areas_)[1]
areas__ = zeros(Int64, (s, 2))
for i = 1 : s
    areas__[i, :] = Int64.(areas_[i][3:4])
end 
GC.gc(true)

In [6]:
scatter(areas__[:, 1], areas__[:, 2], xlabel="Readers' roi", ylabel="Prediction", title="BAC area")
plot!(areas__[:, 1], areas__[:, 1], label="y = x", linestyle=:dash)
savefig(joinpath(output_pred_dir, "BAC_area_plot2.png"))

"C:\\Users\\wenbl13\\Desktop\\BAC\\BAC Project\\Output\\BAC_area_plot2.png"

In [30]:
idx = []
# Check for points
for i = 1 : s
    label_area, pred_area = areas__[i, :]
    ratio = label_area / pred_area
    if ratio > 4
        println(i, ": ", ratio)
    else
        push!(idx, i)
    end
end

13: 4.646377979960843
14: 9.021660275676236
16: 6.037864016604818
22: 4.426022706794017
23: 4.451520523660166
24: 4.294606143674917
70: 4.576734811402246


13: 4.646377979960843  
14: 9.021660275676236  
16: 6.037864016604818  
22: 4.426022706794017  
23: 4.451520523660166  
24: 4.294606143674917  
70: 4.576734811402246  

In [34]:
areas_output = areas__[idx]
df = DataFrame(X = areas__[:, 1], Y = areas__[:, 2])

# Save the DataFrame to a CSV file
CSV.write("BAC_plot_coordinates.csv", df)

193-element Vector{Int64}:
 46550
 50157
 78838
 76932
 34529
 49156
 29689
 82400
 22467
 48105
     ⋮
 50851
 71300
 66677
 92647
 93028
 38315
 57761
 93617
 72387

In [9]:
areas_[13]

4-element Vector{Any}:
       "SID-103073"
       "1.2.840.113681.2230565208.1099.3632750534.308.jld"
 242067.0f0
  52098.0f0

# 4. Save images as png

In [10]:
plot_SIDs = readdir(output_pred_dir)
for SID in plot_SIDs
    curr_dir = joinpath(output_pred_dir, SID)
    if isdir(curr_dir)
        for f in readdir(curr_dir)
            # setup path
            f_name, _ = splitext(f)
            image_path = joinpath(valid_data_dir, "image", SID, f_name*".dcm")
            save_path = joinpath(output_images_dir, SID, "image_"*f_name*".png")
            # read
            dcm_img = Float32.(dcm_parse(image_path)[(0x7fe0, 0x0010)])
            dcm_img = my_process_img(dcm_img)
            dcm_img = Gray.(dcm_img)
            save
            save(save_path, dcm_img)
        end 
    end
end
GC.gc(true)

UndefVarError: UndefVarError: `my_process_img` not defined

In [11]:
function my_process_img(img)
    # get mask
    mask = zoom_pxiel_values(img)
    mask = 1f0 .- round.(mask)
    img_ = img[mask .== 1f0]
    # zoom pixel values to 0 and 1
    a, b = maximum(img_), minimum(img_)
    img_ = (img_ .- b) ./ (a - b)
    img_ = 1f0 .- img_
    # histogram equalization
    img_ = histogram_equalization_(img_)
    img[mask .== 1f0] .= img_
    # save
    img = img .* mask
    return img
end

my_process_img (generic function with 1 method)

In [12]:
function histogram_equalization_(img)
    # Get array length
    len = length(img)
    
    # Initialize histogram and cumulative histogram
    nbins = 256
    hist = zeros(Int, nbins)
    chist = zeros(Int, nbins)
    
    # Compute the histogram
    for val in img
        bin = Int(floor(val * (nbins - 1)) + 1)
        hist[bin] += 1
    end
    
    # Compute the cumulative histogram
    chist[1] = hist[1]
    for i in 2:nbins
        chist[i] = chist[i - 1] + hist[i]
    end
    
    # Perform histogram equalization
    min_chist = minimum(filter(x -> x > 0, chist))
    total_pixels = len
    new_img = zeros(Float32, len)
    
    for i in 1:len
        bin = Int(floor(img[i] * (nbins - 1)) + 1)
        new_intensity = (chist[bin] - min_chist) / (total_pixels - min_chist)
        new_img[i] = Float32(new_intensity)
    end
    
    return new_img
end

histogram_equalization_ (generic function with 1 method)

In [17]:
# img = Float32.(dcm_parse(raw"F:\collected_dataset_for_ML\valid\image\SID-100809\1.2.840.113681.2206989178.1090.3561033058.289.dcm")[(0x7fe0, 0x0010)])
# mask = 1f0 .- zoom_pxiel_values(img)

# s = size(img)

# x = ceil(Int, s[1]/patch_size) + floor(Int, (s[1]-patch_size_half)/patch_size)
# y = ceil(Int, s[2]/patch_size) + floor(Int, (s[2]-patch_size_half)/patch_size)
# locations = []
# ct = 0
# for i = 1 : x-1
#     x_start = 1+(i-1)*patch_size_half
#     x_end = x_start+patch_size-1
#     for j = 1 : y-1
#         y_start = 1+(j-1)*patch_size_half
#         y_end = y_start+patch_size-1
#         # check patch
#         if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
#             # save patch
#             ct += 1
#             push!(locations, [x_start, x_end, y_start, y_end])
#             # img_patches[:, :, 1, ct] = (img[x_start:x_end, y_start:y_end])
#             # locations[ct] = (x_start, x_end, y_start, y_end)
#         end
#     end
#     # right col
#     y_start, y_end = s[2]-patch_size+1, s[2]
#     # check patch
#     if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
#         # save patch
#         ct += 1
#         push!(locations, [x_start, x_end, y_start, y_end])
#         # img_patches[:, :, 1, ct] = (img[x_start:x_end, y_start:y_end])
#         # locations[ct] = (x_start, x_end, y_start, y_end)
#     end
# end
# # last row
# x_start, x_end = s[1]-patch_size+1, s[1]
# for j = 1 : y-1
#     y_start = 1+(j-1)*patch_size_half
#     y_end = y_start+patch_size-1
#     # check patch
#     if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
#         # save patch
#         ct += 1
#         push!(locations, [x_start, x_end, y_start, y_end])
#         # img_patches[:, :, 1, ct] = (img[x_start:x_end, y_start:y_end])
#         # locations[ct] = (x_start, x_end, y_start, y_end)
#     end
# end
# # right col
# y_start, y_end = s[2]-patch_size+1, s[2]
# # check patch
# if mean(mask[x_start:x_end, y_start:y_end]) > 0.35
#     # save patch
#     ct += 1
#     push!(locations, [x_start, x_end, y_start, y_end])
#     # img_patches[:, :, 1, ct] = (img[x_start:x_end, y_start:y_end])
#     # locations[ct] = (x_start, x_end, y_start, y_end)
# end
# s

In [18]:
# img_tmp = Gray.(1f0 .- zoom_pxiel_values(img))
# img_ = RGB.(img_tmp)
# for box in locations
#     x_start, x_end, y_start, y_end = box
#     color = RGB(rand(), rand(), rand())

#     img_[x_start, y_start:y_end] .= color  # Left vertical line
#     img_[x_end, y_start:y_end] .= color   # Right vertical line
#     img_[x_start:x_end, y_start] .= color  # Top horizontal line
#     img_[x_start:x_end, y_end] .= color    # Bottom horizontal line
# end
# save("image_patch_sample.png", img_)